In [18]:
import itertools
import random

In [99]:
def filter_support(data, candidates, min_supp):
    count_dict = {}
    for transaction in data:
        for candidate in candidates:
            if not candidate in count_dict:
                count_dict[candidate] = 0.0
            if candidate.issubset(transaction):
                count_dict[candidate] += 1
    data_size = len(data)
    filtered_candidates = set()
    support_values = {}
    for candidate in candidates:
        support = count_dict[candidate] / data_size
        if support >= min_supp:
            filtered_candidates.add(candidate)
            support_values[candidate] = support
    return filtered_candidates, support_values

def generate_candidates(prev_candidates):
    k = len(random.sample(prev_candidates, 1)[0])
    new_candidates = set()
    for canA, canB in itertools.combinations(prev_candidates, 2):
        listA, listB = sorted(list(canA))[:-1], sorted(list(canB))[:-1]
        if listA == listB:
            new_candidates.add(canA | canB)
    return new_candidates

def generateL1(data, min_supp):
    unique_elements = {frozenset([elem]) for transaction in data for elem in transaction}
    print "Unique elements created:", len(unique_elements)
    return filter_support(data, unique_elements, min_supp)

def apriori(data, min_supp):
    L1, all_support_values = generateL1(data, min_supp)
    print "Generated L1"
    L = [L1]
    k = 2
    while L[k-2]:
        candidates = generate_candidates(L[k-2])
        filtered_candidates, support_values = filter_support(data, candidates, min_supp)
        print "Done step", k-1
        all_support_values.update(support_values)
        L.append(filtered_candidates)
        k += 1
    all_support_values.update({frozenset([]): 1.0})
    return reduce(lambda x, y: x | y, L), all_support_values 

def generate_reliable_rules(frequent_sets, support_values, min_confidence):
    reliable_rules = set()
    for set_ in frequent_sets:
        S = [set([(set_, frozenset())])]
        k = 1
        while S[k-1]:
            Sk = set()
            for rule in S[-1]:
                left, right = rule
                for elem in left:
                    left_copy = set(left)
                    left_copy.remove(elem)
                    confidence = support_values[set_] / support_values[frozenset(left_copy)]
                    if confidence >= min_confidence:
                        right_copy = set(right)
                        right_copy.add(elem)
                        Sk.add((frozenset(left_copy), frozenset(right_copy)))
            S.append(Sk)
            k += 1
        reliable_rules |= reduce(lambda x, y: x | y, S)
    return reliable_rules

def print_rules(rules, support_values, min_lift=1.0, min_leverage=0.0):
    
    def print_set(set_):
        set_string = "{"
        for e in set_:
            set_string += (str(e) + ", ")
        if set_string[-1] != "{":
            set_string = set_string[:-2]
        return set_string + "}"
    
    for left, right in rules:
        confidence = support_values[left | right] / support_values[left]
        lift = confidence / support_values[right]
        leverage = support_values[left | right] - support_values[left]*support_values[right]
        if lift >= min_lift and leverage >= min_leverage:
            print print_set(left) + " => " + print_set(right) + ",",
            print "confidence: " + str(confidence) + ",",
            print "lift: " + str(lift) + ",",
            print "leverage: " + str(leverage)

# Problem 1

In [104]:
def generate_random_dataset(size, max_items):
    return {frozenset(random.sample(range(max_items), random.randint(1, max_items))) for _ in range(size)}

In [113]:
random_dataset = generate_random_dataset(10000, 15)
modified_dataset = {t|frozenset([1, 2]) if random.random() < 0.75 else t for t in random_dataset}
freq_sets = apriori(modified_dataset, 0.7)
print "Frequent sets:", freq_sets[0]

Unique elements created: 15
Generated L1
Done step 1
Done step 2
Frequent sets: set([frozenset([1, 2]), frozenset([2]), frozenset([1])])


In [114]:
rules = generate_reliable_rules(freq_sets[0], freq_sets[1], 0.5)
print_rules(rules, freq_sets[1], min_leverage=0.01)

{1} => {2}, confidence: 0.906862745098, lift: 1.0786099039, leverage: 0.0560314639148
{2} => {1}, confidence: 0.91441207076, lift: 1.0786099039, leverage: 0.0560314639148


# Problem 2

In [55]:
def read_dataset(filename):
    data = set()
    with open(filename, "r") as f:
        for line in f:
            if line != "\n":
                data.add(frozenset(map(int, line.split())))
    return data

In [56]:
dataset = read_dataset("retail.dat")
print "Data loaded"
frequent_sets, support_values = apriori(dataset, 0.01)
print "#frequent_sets:", len(frequent_sets)

Data loaded
Unique elements created: 16470
Generated L1
Done step 1
Done step 2
Done step 3
Done step 4
#frequent_sets: 169


In [103]:
rules = generate_reliable_rules(frequent_sets, support_values, 0.5)
print_rules(rules, support_values, min_lift=1.3, min_leverage=0.02)

{36} => {38}, confidence: 0.95337911095, lift: 5.39754675346, leverage: 0.0257427170229
{48, 41} => {39}, confidence: 0.815777627729, lift: 1.42309390178, leverage: 0.0258134972152
{41, 39} => {48}, confidence: 0.652886607223, lift: 1.33798485118, leverage: 0.0219325777492
{170} => {38}, confidence: 0.981362250085, lift: 5.55597302906, leverage: 0.0284436359895
{110} => {38}, confidence: 0.973408541499, lift: 5.51094318368, leverage: 0.0236866636107
{41} => {39}, confidence: 0.762097604503, lift: 1.3294510865, leverage: 0.0329552385461


# Problem 3

In [ ]:
kosarak = read_dataset("kosarak.dat")
print "Data loaded"

In [ ]:
kosarak_frequent_sets1, kosarak_support_values1 = apriori(kosarak, 0.01)
print "#frequent_sets (1%):", len(kosarak_frequent_sets1)

In [ ]:
kosarak_frequent_sets05, kosarak_support_values05 = apriori(kosarak, 0.005)
print "#frequent_sets (0.5%):", len(kosarak_frequent_sets05)